# Running the comparator between developmental stages in Zepp et. al.

In [ ]:
%%time
%load_ext autoreload
%autoreload 2

import os
import warnings


import pandas as pd
import scanpy as sc

from sklearn.exceptions import ConvergenceWarning

from gepdynamics import _utils
from gepdynamics import comparator

# Move to the project's home directory, as defined in _constants
_utils.cd_proj_home()
print(os.getcwd())

/cs/labs/mornitzan/yotamcon/gep-dynamics
CPU times: user 3.48 s, sys: 1.71 s, total: 5.2 s
Wall time: 1min 45s


In [ ]:
%%time
results_dir = _utils.set_dir('results_zepp')

split_adatas_dir = results_dir.joinpath('split_adatas')

stages = ['E12', 'E15', 'E17', 'P3', 'P7', 'P15', 'P42']

split_adatas = {stage: sc.read_h5ad(split_adatas_dir.joinpath(f'{stage}_GEPs.h5ad')) for stage in stages}

CPU times: user 2.29 s, sys: 2.25 s, total: 4.54 s
Wall time: 27.4 s


In [21]:
for stage in stages:
    tmp = split_adatas[stage]
    # pd.Categorical(adata.obs['timesimple'].map(
    #    dict(zip(pd.Categorical(adata.obs['timesimple']).categories, adata.uns['timesimple_colors']))))
    field_1 = 'compartment'
    field_2 = 'celltype'
    tmp.obsm['row_colors'] = pd.concat([
        tmp.obs[field_1].map(tmp.uns[f'{field_1}_colors_dict']),
        tmp.obs[field_2].map(tmp.uns[f'{field_2}_colors_dict'])], axis=1)

In [48]:
%%time

pairs = [(stages[i], stages[i + 1]) for i in range(len(stages) -1)]

for stage_a, stage_b in pairs:
    comparison_dir = _utils.set_dir(results_dir.joinpath(f"{stage_a}_{stage_b}"))
    
    adata_a = split_adatas[stage_a]
    adata_b = split_adatas[stage_b]
    
    if os.path.exists(comparison_dir.joinpath('comparator.npz')):
        cmp = comparator.Comparator.load_from_file(comparison_dir.joinpath('comparator.npz'), adata_a, adata_b)
    else:
        cmp = comparator.Comparator(adata_a, adata_a.obsm['usages'], adata_b, comparison_dir, 'torchnmf', device='cuda', max_nmf_iter=1000, verbosity=1)
        
        print('decomposing')
        cmp.extract_geps_on_jointly_hvgs()
        cmp.decompose_b(repeats = 20)
    
    cmp.examine_adata_a_decomposition_on_jointly_hvgs()
    
    cmp.print_errors()
    cmp.examine_adata_b_decompositions()
    cmp.plot_decomposition_comparisons()
    
    cmp.calculate_fingerprints()
    
    print('running GSEA')
    cmp.run_gsea(gprofiler_kwargs=dict(organism='mmusculus', sources=['GO:BP', 'WP', 'REAC', 'KEGG', 'TF', 'HP']))

    cmp.save_to_file(comparison_dir.joinpath('comparator.npz'))

    

dn_16 error =  5114335.6
dn_17 error =  5084008.0
dn_18 error =  5057438.8
dn_19 error =  5021209.0
E12 error =  6034421.6
E12e1 error =  5781610.4
E12e2 error =  5660207.4
E12e3 error =  5529197.7
Number of edges=464
Number of edges=1500
running GSEA
Directory "results_zepp/E15_E17" does not exist.
trying to create it at /cs/labs/mornitzan/yotamcon/gep-dynamics/results_zepp/E15_E17
decomposing
Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 4673905.607779528, # iterations was 90
Decomposing B de-novo
beta 1 loss = 3985083.457523105, # iterations was 500
beta 1 loss = 3938755.9288857915, # iterations was 350
beta 1 loss = 3918052.0249028774, # iterations was 490
beta 1 loss = 3897030.6062946157, # iterations was 370
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 66 iterations reached error =  4435692.4
repeat 3, after 66 iterations reached error =  4435685.8
repeat 4, af

In [46]:
for stage_a, stage_b in pairs:
    comparison_dir = results_dir.joinpath(f"{stage_a}_{stage_b}", 'comparator.npz')
    
    adata_a = split_adatas[stage_a]
    adata_b = split_adatas[stage_b]
    
    cmp = comparator.Comparator.load_from_file(comparison_dir, adata_a, adata_b)
    break

### Epithelial only comparator objects

In [ ]:
%%time

epi_split_adatas_dir = results_dir.joinpath('epi_split_adatas')

stages = ['E12', 'E15', 'E17', 'P3', 'P7', 'P15', 'P42']

epi_split_adatas = {stage: sc.read_h5ad(epi_split_adatas_dir.joinpath(f'epi_{stage}_GEPs.h5ad')) for stage in stages}

CPU times: user 1.57 s, sys: 567 ms, total: 2.14 s
Wall time: 17 s


In [ ]:
for stage in stages:
    tmp = epi_split_adatas[stage]

    field_1 = 'compartment'
    field_2 = 'celltype'
    tmp.obsm['row_colors'] = pd.concat([
        tmp.obs[field_1].map(tmp.uns[f'{field_1}_colors_dict']),
        tmp.obs[field_2].map(tmp.uns[f'{field_2}_colors_dict'])], axis=1)

In [23]:
%%time

pairs = [(stages[i], stages[i + 1]) for i in range(len(stages) -1)]
pairs = [('P7', 'P15', )]

for stage_a, stage_b in pairs:
    comparison_dir = _utils.set_dir(results_dir.joinpath(f"epi_{stage_a}_{stage_b}"))
    
    adata_a = epi_split_adatas[stage_a]
    adata_b = epi_split_adatas[stage_b]
    
    if os.path.exists(comparison_dir.joinpath('comparator.npz')):
        cmp = comparator.Comparator.load_from_file(comparison_dir.joinpath('comparator.npz'), adata_a, adata_b)
    else:
        cmp = comparator.Comparator(adata_a, adata_a.obsm['usages'], adata_b, comparison_dir, 'sklearn', max_nmf_iter=1000, verbosity=1)
        
        print('decomposing')
        cmp.extract_geps_on_jointly_hvgs()
        cmp.decompose_b(repeats = 20)
    
    cmp.examine_adata_a_decomposition_on_jointly_hvgs()
    
    cmp.print_errors()
    cmp.examine_adata_b_decompositions()
    cmp.plot_decomposition_comparisons()
    
    cmp.calculate_fingerprints()
    
    print('running GSEA')
    cmp.run_gsea(gprofiler_kwargs=dict(organism='mmusculus', sources=['GO:BP', 'WP', 'REAC', 'KEGG', 'TF', 'HP']))

    cmp.save_to_file(comparison_dir.joinpath('comparator.npz'))

decomposing
Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
Epoch 10 reached after 0.082 seconds, error: 1256.033738
Epoch 20 reached after 0.142 seconds, error: 1254.574878
Epoch 30 reached after 0.203 seconds, error: 1254.342756
Epoch 40 reached after 0.262 seconds, error: 1254.275787
Epoch 50 reached after 0.322 seconds, error: 1254.250174
Epoch 60 reached after 0.381 seconds, error: 1254.238414
Epoch 70 reached after 0.443 seconds, error: 1254.232136
Epoch 80 reached after 0.502 seconds, error: 1254.228946
Epoch 90 reached after 0.562 seconds, error: 1254.226704
Epoch 100 reached after 0.624 seconds, error: 1254.225558
Decomposing B de-novo
Epoch 10 reached after 0.109 seconds, error: 1218.704845
Epoch 20 reached after 0.202 seconds, error: 1211.940902
Epoch 30 reached after 0.291 seconds, error: 1209.424760
Epoch 40 reached after 0.379 seconds, error: 1208.058256
Epoch 50 reached after 0.470 seconds, error: 1207.171280
Epoch 60 

In [50]:
for stage_a, stage_b in pairs:
    comparison_dir = _utils.set_dir(results_dir.joinpath(f"epi_{stage_a}_{stage_b}"))
    
    adata_a = epi_split_adatas[stage_a]
    adata_b = epi_split_adatas[stage_b]
    
    cmp = comparator.Comparator.load_from_file(comparison_dir, adata_a, adata_b)
    break

In [21]:
var_names_gt_2 = sc.pp.filter_genes(epi_split_adatas['E15'].X, min_cells=2)[0]
np.sum(var_names_gt_2)

15669

In [18]:
cmp

Comparator(adata_a=P15, adata_b=P42) at stage Decomposed. engine=sklearn.